# **LangChain Templating: Syntax and Examples**

## **📌Overview**
Prompt templates allow for the dynamic generation of prompts by inserting variables into a structured format. \
This enables the creation of flexible and reusable prompts for different tasks.

LangChain supports multiple **templating syntaxes**, including (but not limited to):

- **Basic String Formatting (`{variable}`)**  
- **Multiple Placeholders in a Single Template (`{var1}, {var2}`)**  
- **Tuple-Based Formatting (`("system", "text {var}")`)**  
- **Using `HumanMessage` Objects (`HumanMessage(content="text")`)**  

In [2]:
# Install dependencies 
%pip install -q python-dotenv langchain-core langchain-google-genai langchain
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install -q google-generativeai==0.8.4 google-ai-generativelanguage==0.6.15

Note: you may need to restart the kernel to use updated packages.
Found existing installation: google-ai-generativelanguage 0.6.16
Uninstalling google-ai-generativelanguage-0.6.16:
  Successfully uninstalled google-ai-generativelanguage-0.6.16
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.0 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

## **Obtain a Google Gemini API Key (GOOGLE COLLAB SETUP):**

If you have a Google Gemini API Key: 
- Copy your API key and replace "your_google_api_key_here" in the code below

Otherwise:  
- Go to the Google AI Studio API Console: [Google AI Studio](https://aistudio.google.com/prompts/new_chat)
- Sign in with your Google account and create a new API key.
- Copy your API key and replace "your_google_api_key_here" in the code below

In [ ]:
# Set your Google API key manually
import os
os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"

## **Load Environment Variables (LOCAL SETUP)**

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

---

## **Basic String Formatting**  

Uses LangChain's built-in `{}` placeholders to dynamically insert values into prompts.  

In [5]:
# Basic string 
template = "Tell me an interesting fact about {object}"
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.invoke({"object": "black holes"})
print(prompt)

messages=[HumanMessage(content='Tell me an interesting fact about black holes', additional_kwargs={}, response_metadata={})]


## **Multiple Placeholders in a Single Template**  

Supports multiple dynamic placeholders within a single prompt template.  

In [6]:
# Multiple placeholders
template_multiple = """You are a convincing essay writer.
Human: Write an essay on why {subject} {position}.
Writer:"""
prompt_multiple = ChatPromptTemplate.from_template(template_multiple)

prompt = prompt_multiple.invoke({"subject": "Gorillas", "position": "are a reasonable pet to have in the household"})
print(prompt)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = llm.invoke(prompt)
print(result.content)

messages=[HumanMessage(content='You are a convincing essay writer.\nHuman: Write an essay on why Gorillas are a reasonable pet to have in the household.\nWriter:', additional_kwargs={}, response_metadata={})]
The notion of a gorilla as a household pet is, to put it mildly, unconventional.  However, a closer examination, devoid of emotional bias and firmly grounded in pragmatic considerations, reveals that with sufficient resources, meticulous planning, and a profound understanding of primate behavior, a gorilla could, theoretically, be a reasonable—though undeniably challenging—pet.  Let’s explore this seemingly absurd proposition.

Firstly, the argument rests on the premise of responsible pet ownership amplified exponentially.  We wouldn’t consider a goldfish without a tank; similarly, a gorilla necessitates a habitat of immense scale.  A custom-built enclosure, mimicking the gorilla’s natural environment, would be essential.  This would involve significant land area, specialized clim

## **Tuple-Based Formatting**
Uses tuples to format system and human messages dynamically in multi-turn conversations.

In [ ]:
# Tuple-based formatting
messages = [
    ("system", "You are a world travel expert who provides insights on {destination}."),
    ("human", "What are the top {activity} experiences in {destination}?"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"destination": "Tokyo", "activity": "food"})
print(prompt)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# result = llm.invoke(prompt)
# print(result.content)

messages=[SystemMessage(content='You are a world travel expert who provides insights on Tokyo.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the top food experiences in Tokyo?', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content="You are an expert in art. Your task is to explain concepts related to this field.\n        If the concept is directly related to art, provide a simple explanation.\n        If the concept is unrelated, respond with: 'I'm sorry, but neural networks is outside my expertise in art.'", additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain neural networks in simple terms.', additional_kwargs={}, response_metadata={}), AIMessage(content="Sure! Here's a breakdown of neural networks.", additional_kwargs={}, response_metadata={})]


## **Using HumanMessage Objects**
Allows structured human input messages within a LangChain chat prompt.

In [ ]:
# HumanMessage objects
messages = [
    ("system", "You are a tour guide specializing in {location}."),
    HumanMessage(content="Describe three must-visit places."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"location": "Halifax"})
print(prompt)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# result = llm.invoke(prompt)
# print(result.content)

messages=[SystemMessage(content='You are a tour guide specializing in Halifax.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Describe three must-visit places.', additional_kwargs={}, response_metadata={})]


---

## **Templating Example**

In [4]:
# Extended example of how to start using templating 
args_dictionary = {"field": "", "concept": "", "difficulty": ""}
valid_difficulties = {"basic", "intermediate", "advanced"}

# Define prompt template
messages = [
    ("system", """You are an expert in {field}. Your role is to explain concepts clearly at the appropriate difficulty level.
    - If the concept is relevant to {field}, provide a clear explanation at the requested difficulty.
    - If the concept is unrelated, respond with: "I'm sorry, but {concept} is outside my expertise in {field}." """),
    
    ("human", "Explain {concept} in {difficulty} terms."),    
    
    ("ai", "Sure! Here's a breakdown of {concept} at the {difficulty} level.")
]
prompt_template = ChatPromptTemplate.from_messages(messages)

# Get user inputs
field = input("What field do you want to learn about? ").strip()
while not field: 
    print("!Error!: Field cannot be empty.")
    field = input("What field do you want to learn about? ").strip()

concept = input("What concept would you like to learn about: ").strip()
while not concept:
    print("!Error!: Field cannot be empty.")
    concept = input("What concept would you like to learn about: ").strip()

difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
while difficulty not in valid_difficulties: 
    print("Error: Please enter a valid difficulty level (basic, intermediate, advanced).")
    difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
    
print("Input Accepted!")
print(f"- Field: {field}")
print(f"- Concept: {concept}")
print(f"- Difficulty: {difficulty}\n")

# Update dictionary with user input
args_dictionary.update({
    "field": field,
    "concept": concept,
    "difficulty": difficulty
})

# Invoke prompt template 
prompt = prompt_template.invoke(args_dictionary)

# Get LLM response
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = llm.invoke(prompt)
print(result.content)

Input Accepted!
- Field: phsyics
- Concept: computers
- Difficulty: basic

A computer is a machine that can accept information (input), process it according to a set of instructions (program), and produce results (output).  Think of it like a super-fast, incredibly obedient calculator that can do much more than just arithmetic.

Here's a simplified breakdown:

* **Input:** This is the information you give the computer.  Examples include typing on a keyboard, clicking a mouse, or speaking into a microphone.

* **Processing:** This is where the "magic" happens.  The computer's central processing unit (CPU), often called the "brain" of the computer, follows instructions from a program to manipulate the input data.  These instructions might involve calculations, sorting information, or displaying images.  The CPU does this by using its memory to store and retrieve information quickly.

* **Output:** This is the result of the processing. Examples include text displayed on a screen, sounds f